In [1]:
import tensorflow as tf
import numpy as np
from numpy.linalg import norm
import mediapipe as mp
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<img src="https://i.imgur.com/3j8BPdc.png" style="height: 300px">

In [3]:
def calcAngle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (norm(ba) * norm(bc))
    radians = np.arccos(cosine)
    degrees = (180/np.pi)*radians
    return int(degrees)

In [4]:
def writeText(img, text, loc, color, size=1):
    cv2.putText(
        img, 
        text, 
        loc,
        cv2.FONT_HERSHEY_SIMPLEX, 
        size, 
        color, 
        2, 
        cv2.LINE_AA
    )


In [5]:
cap = cv2.VideoCapture(0)
state = "down"
count = 0

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break
        
    img_height, img_width = img.shape[:2]
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        detections = pose.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    shoulder_idx = mp_pose.PoseLandmark.LEFT_SHOULDER.value
    elbow_idx = mp_pose.PoseLandmark.LEFT_ELBOW.value
    wrist_idx = mp_pose.PoseLandmark.LEFT_WRIST.value
    
    if detections.pose_landmarks is not None:
        landmarks = detections.pose_landmarks.landmark
        shoulder = landmarks[shoulder_idx]
        elbow = landmarks[elbow_idx]
        wrist = landmarks[wrist_idx]
        angle = calcAngle((shoulder.x, shoulder.y), (elbow.x, elbow.y), (wrist.x, wrist.y))
        writeText(img, str(angle), tuple(np.multiply([elbow.x, elbow.y], [img_width, img_height]).astype(int)), (255, 0, 0))
        
        if angle < 50 and state != "up":
            state = "up"
            count += 1
        if angle > 120:
            state = "down"
    
    cv2.rectangle(img, (0, 0), (200, 80), (255, 0, 0), -1)
    writeText(img, "Reps: ", (10, 25), (255, 255, 255))
    writeText(img, str(count), (110, 25), (255, 255, 255), 1)
    
    writeText(img, "State: ", (10, 60), (255, 255, 255))
    writeText(img, state, (110, 60), (255, 255, 255), 1)
    
    mp_drawing.draw_landmarks(
        image=img,
        landmark_list=detections.pose_landmarks,
        connections=mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
    )
    
    cv2.imshow('mediapipe', img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()